# FLASHCARD GENERATOR RAG MODEL (Open Source)

In [1]:
import tools.pipeline

c:\Users\schla\Desktop\Gitlab_Repositories\master-thesis-ai-meets-classroom\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### DATA PREPARATION

In [2]:
data = tools.pipeline.load_wikipedia_dataset()

INFO:tools.pipeline:Loading Wikipedia dataset: language=simple, date=20220301


TextCharacter Splitting & Providing Chunks

In [3]:
texts, chunk_counts, total_chunks, total_valid_chunks = tools.pipeline.process_documents(data)

INFO:tools.pipeline:Processing documents into chunks
INFO:tools.pipeline:Total chunks: 216232, Valid chunks: 179456


#### DATA STORAGE (ChromaDB, Nomic Embeddings)

In [4]:
#vectorestore, embeddings = tools.pipeline.setup_vectorstore_nomic(texts)

In [4]:
vectorstore, embeddings = tools.pipeline.load_vectorstore_nomic()

INFO:tools.pipeline:Loading vector store
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:chromadb.api.segment:Collection opensource_rag_wikipedia is not created.
INFO:tools.pipeline:Vector store loaded successfully


#### RETRIEVER AND GENERATOR (Llama 3.1 Instruct Turbo)

In [5]:
topic = "Correlation"

In [6]:
flashcard_set = tools.pipeline.generate_question_answer_pairs_open_source_json(topic, vectorstore, threshold = 0.8)

INFO:tools.pipeline:Generating QA pairs
INFO:tools.pipeline:Retrieving documents for topic: Correlation with threshold: 0.8
INFO:tools.pipeline:Formatting documents
ERROR:tools.pipeline:Failed to parse JSON response
ERROR:tools.pipeline:Failed to parse JSON response
ERROR:tools.pipeline:Failed to parse JSON response
ERROR:tools.pipeline:Failed to parse JSON response
INFO:tools.pipeline:Generated a total of 39 QA pairs


In [7]:
print(flashcard_set)

[
    {
        "question": "What does correlation measure in statistics and probability theory?",
        "answer": "How closely related two sets of data are.",
        "context": "In statistics and probability theory, correlation is a way to indicate how closely related two sets of data are.",
        "source": "https://simple.wikipedia.org/wiki/Correlation"
    },
    {
        "question": "What is an important consideration when interpreting the relationship between two correlated variables?",
        "answer": "A third factor may be involved, and correlation does not necessarily imply causation.",
        "context": "Correlation does not always mean that one causes the other. In fact, it is very possible that there is a third factor involved.",
        "source": "https://simple.wikipedia.org/wiki/Correlation"
    },
    {
        "question": "What happens to two sets of data when they have a positive correlation?",
        "answer": "They go up together.",
        "context": "Corr

### EVALUATION: RETRIEVER

In [8]:
dataframe_retriever_evaluation = tools.pipeline.evaluate_retriever(topic, vectorstore)
dataframe_retriever_evaluation

INFO:tools.pipeline:Retriever evaluation complete


,ID,URL,Context,Similarity Score,Distance Score
0,64761,https://simple.wikipedia.org/wiki/Correlation,"In statistics and probability theory, correlat...",0.867254,0.132746
1,804693,https://simple.wikipedia.org/wiki/Pearson%20pr...,Pearson's correlation is a mathematical formul...,0.812699,0.187301
2,187866,https://simple.wikipedia.org/wiki/Spearman%27s...,"In mathematics and statistics, Spearman's rank...",0.801714,0.198286
3,531764,https://simple.wikipedia.org/wiki/Rank%20corre...,A rank correlation is any statistic that measu...,0.800087,0.199913
4,342118,https://simple.wikipedia.org/wiki/Denying%20th...,Denying the correlative is a mistake in logic ...,0.686136,0.313864
5,470230,https://simple.wikipedia.org/wiki/Statistical%...,Statistics uses variables to describe a measur...,0.660952,0.339048
6,76245,https://simple.wikipedia.org/wiki/Conjunction%...,Conjunction is a word used in astronomy and as...,0.654611,0.345389
7,109208,https://simple.wikipedia.org/wiki/Pattern%20ma...,Pattern matching is a family of tools for find...,0.650239,0.349761
8,821533,https://simple.wikipedia.org/wiki/Serial%20pos...,The serial position effect is a psychological ...,0.644498,0.355502
9,472068,https://simple.wikipedia.org/wiki/Regression%2...,"Regression toward the mean simply means that, ...",0.643775,0.356225


In [9]:
dataset = tools.pipeline.prepare_dataset(flashcard_set)
tools.pipeline.save_qa_pairs(dataset, folder_name="data_flashcard_generator", topic=topic, pipeline_name="flashcard_generator",content="generated_qas")

INFO:tools.pipeline:Preparing dataset from JSON string
Casting the dataset: 100%|██████████| 39/39 [00:00<00:00, 5260.58 examples/s]
INFO:tools.pipeline:Dataset preparation complete
INFO:tools.pipeline:Saving QA pairs to CSV: data_flashcard_generator/generated_qas_Correlation_flashcard_generator.csv
INFO:tools.pipeline:File saved as CSV.


### EVALUATION: GENERATOR

⚡MANUAL STEP is required to add handwritten ground truths, or the evaluation is performed without them. However, not all evaluation metrics can be calculated.

In [10]:
dataset_gt = tools.pipeline.load_qa_pairs(folder_name="data_flashcard_generator", topic=topic, pipeline_name="flashcard_generator", content="generated_qas")
dataset_gt = dataset_gt[:3]
dataset_gt

INFO:tools.pipeline:Loading QA pairs from CSV: data_flashcard_generator/generated_qas_Correlation_flashcard_generator.csv
INFO:tools.pipeline:QA pairs loaded successfully


,question,answer,contexts,source
0,What does correlation measure in statistics an...,How closely related two sets of data are.,"In statistics and probability theory, correlat...",https://simple.wikipedia.org/wiki/Correlation
1,What is an important consideration when interp...,"A third factor may be involved, and correlatio...",Correlation does not always mean that one caus...,https://simple.wikipedia.org/wiki/Correlation
2,What happens to two sets of data when they hav...,They go up together.,Correlation usually has one of two directions....,https://simple.wikipedia.org/wiki/Correlation


### RAGAS Metrics

In [11]:
ragas_data = tools.pipeline.ragas_prepare_data(dataset = dataset_gt, ground_truth=False)

Map: 100%|██████████| 3/3 [00:00<00:00, 505.81 examples/s]


Casting the dataset: 100%|██████████| 3/3 [00:00<00:00, 641.76 examples/s]
INFO:tools.pipeline:Data prepared for Ragas evaluation


In [12]:
ragas_data

Dataset({
    features: ['question', 'answer', 'contexts', 'source'],
    num_rows: 3
})

In [13]:
ragas_evaluation_data = tools.pipeline.ragas_evaluate_data(ragas_data, ground_truth=False)
ragas_evaluation_data

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Evaluating:  17%|█▋        | 1/6 [00:03<00:18,  3.71s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Requ

{'faithfulness': 1.0000, 'answer_relevancy': 0.9353}

In [14]:
tools.pipeline.save_qa_pairs(ragas_evaluation_data, folder_name = "data_flashcard_generator", topic = topic, pipeline_name = "flashcard_generator", content="ragas_metrics")

INFO:tools.pipeline:Saving QA pairs to CSV: data_flashcard_generator/ragas_metrics_Correlation_flashcard_generator.csv
INFO:tools.pipeline:File saved as CSV.


### DeepEval Metrics

In [15]:
deepeval_data = tools.pipeline.deepeval_prepare_data(dataset_gt, ground_truth=False)

INFO:tools.pipeline:Preparing data for DeepEval evaluation
INFO:tools.pipeline:Data prepared for DeepEval evaluation


In [16]:
tools.pipeline.deepeval_evaluate_data(deepeval_data, ground_truth=False)

INFO:tools.pipeline:Starting DeepEval evaluation


✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-4, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Bias Metric! (using gpt-4, strict=False, async_mode=True)...

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 3 test case(s) in parallel: |          |  0% (0/3) [Time Taken: 00:00, ?test case/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200



Metrics Summary

  - ✅ Faithfulness (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4, reason: The score is 1.00 because there are no contradictions between the actual output and the retrieval context. Great job!, error: None)
  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4, reason: The score is 1.00 because the output precisely addresses the input query about positive correlation between two sets of data, without any irrelevant statements., error: None)
  - ✅ Contextual Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4, reason: The score is 1.00 because the input was perfectly matched with no irrelevancy found., error: None)
  - ✅ Bias (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4, reason: The score is 0.00 because there were no instances of bias identified in the actual output., error: None)
  - ✅ Toxicity (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4

✓ Tests finished 🎉! Run 'deepeval login' to view evaluation results on Confident AI. 
‼️  NOTE: You can also run evaluations on ALL of deepeval's metrics directly on Confident AI instead.

INFO:tools.pipeline:DeepEval evaluation complete


### Haystack Evaluation

In [17]:
haystack_data = tools.pipeline.haystack_prepare_data(dataset_gt, ground_truth=False)

INFO:tools.pipeline:Preparing data for Haystack evaluation
INFO:tools.pipeline:Data prepared for Haystack evaluation


In [18]:
haystack_data

[{'questions': ['What does correlation measure in statistics and probability theory?'],
  'predicted_answers': ['How closely related two sets of data are.'],
  'contexts': ['In statistics and probability theory, correlation is a way to indicate how closely related two sets of data are.']},
 {'questions': ['What is an important consideration when interpreting the relationship between two correlated variables?'],
  'predicted_answers': ['A third factor may be involved, and correlation does not necessarily imply causation.'],
  'contexts': ['Correlation does not always mean that one causes the other. In fact, it is very possible that there is a third factor involved.']},
 {'questions': ['What happens to two sets of data when they have a positive correlation?'],
  'predicted_answers': ['They go up together.'],
  'contexts': ['Correlation usually has one of two directions. These are positive or negative. If it is positive, then the two sets go up together. If it is negative, then one goes u

In [19]:
df_haystack_evaluation_data = tools.pipeline.haystack_evaluate_data(haystack_data, ground_truth=False)

INFO:tools.pipeline:Starting Haystack evaluation
INFO:haystack.core.pipeline.pipeline:Running component context_relevance_evaluator
100%|██████████| 1/1 [00:02<00:00,  2.33s/it]
INFO:haystack.core.pipeline.pipeline:Running component faithfulness_evaluator
100%|██████████| 1/1 [00:00<00:00,  1.27it/s]
INFO:haystack.core.pipeline.pipeline:Running component context_relevance_evaluator
100%|██████████| 1/1 [00:00<00:00,  1.46it/s]
INFO:haystack.core.pipeline.pipeline:Running component faithfulness_evaluator
100%|██████████| 1/1 [00:00<00:00,  1.15it/s]
INFO:haystack.core.pipeline.pipeline:Running component context_relevance_evaluator
100%|██████████| 1/1 [00:00<00:00,  1.60it/s]
INFO:haystack.core.pipeline.pipeline:Running component faithfulness_evaluator
100%|██████████| 1/1 [00:00<00:00,  1.34it/s]
INFO:tools.pipeline:Overall Scores: {'context_relevance_score': 1, 'faithfulness_score': 0.0}


In [20]:
df_haystack_evaluation_data

,question,predicted_answer,context,context_relevance_score,faithfulness_score
0,What does correlation measure in statistics an...,How closely related two sets of data are.,"In statistics and probability theory, correlat...",1,1.0
1,What is an important consideration when interp...,"A third factor may be involved, and correlatio...",Correlation does not always mean that one caus...,1,1.0
2,What happens to two sets of data when they hav...,They go up together.,Correlation usually has one of two directions....,1,0.0


In [21]:
tools.pipeline.save_qa_pairs(df_haystack_evaluation_data, folder_name="data_flashcard_generator", topic=topic, pipeline_name="flashcard_generator", content="haystack_metrics")

INFO:tools.pipeline:Saving QA pairs to CSV: data_flashcard_generator/haystack_metrics_Correlation_flashcard_generator.csv
INFO:tools.pipeline:File saved as CSV.
